In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/Headsets/data.zip /content

In [ ]:
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
import os
import random
import shutil
from pathlib import Path

source_path = Path('/content/custom_data')
dest_path = Path('/content/data')
split_ratio = 0.9

def setup_dataset():
    for split in ['train', 'validation']:
        for folder in ['images', 'labels']:
            (dest_path / split / folder).mkdir(parents=True, exist_ok=True)

    images = [f for f in os.listdir(source_path / 'images') if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    random.seed(42)
    random.shuffle(images)

    split_idx = int(len(images) * split_ratio)
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    def move_files(files, split_name):
        for filename in files:
            file_stem = Path(filename).stem

            src_img = source_path / 'images' / filename
            dst_img = dest_path / split_name / 'images' / filename
            shutil.copy(src_img, dst_img)

            label_name = f"{file_stem}.txt"
            src_label = source_path / 'labels' / label_name
            dst_label = dest_path / split_name / 'labels' / label_name

            if src_label.exists():
                shutil.copy(src_label, dst_label)
            else:
                print(f"Warning: Label not found for {filename}")

    print(f"Moving {len(train_images)} files to Train...")
    move_files(train_images, 'train')

    print(f"Moving {len(val_images)} files to Validation...")
    move_files(val_images, 'validation')

    print("Process completed successfully!")

if __name__ == "__main__":
    setup_dataset()

In [ ]:
!pip install ultralytics

In [ ]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yamlimport yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=1024 plots=True patience=20

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')

results = model.predict(source='/content/gdrive/MyDrive/Headsets/pag17_linha14.png', imgsz=1024, conf=0.5)

for r in results:
    boxes = r.boxes
    for box in boxes:
        # Coordinates in pixels (x1, y1, x2, y2)
        # x1, y1 is the upper left corner
        # x2, y2 is the lower right corner
        x1, y1, x2, y2 = box.xyxy[0].tolist()

        # Class detected
        cls = int(box.cls[0])
        class_name = model.names[cls]

        print(f"Detected {class_name} in: X={x1}, Y={y1}")

        # Selenium for clicking or moving the mouse:
        # driver.execute_script(f"window.scrollTo({x1}, {y1});")

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content